In [36]:
import os

import numpy as np
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer

from InputDataset import FramingArticleDataset

In [1]:
import pandas as pd

In [3]:
df = pd.DataFrame({'a': [1, 1, 1], 'b': [2, 2, 2]})

In [6]:
pd.DataFrame(df)

,a,b
0,1,2
1,1,2
2,1,2


# Test tokenizer, lemmatization, stopword and punctuation removal with Spacy

In [70]:
sem_eval_train = FramingArticleDataset(data_dir='data', language='en', subtask=2, split='train')
sem_eval_train.separate_title_content(remove_raw_data_col=False)

433it [00:00, 29338.38it/s]


In [71]:
train_df = sem_eval_train.df

In [128]:
a = en_nlp(train_df.content.loc[711964584])

In [144]:
a = en_nlp(train_df.content.loc[111111114])

In [149]:
next(en_nlp(train_df.content.loc[111111114]).sents, 5)

A U.S. judge on Wednesday ordered the release of an illegal immigrant who is among 47 Indonesians in New Hampshire challenging the Trump administration’s order to deport them.

In [145]:
b = next(a.sents)

In [168]:
def extract_n_sentences(text, nlp, n_sentences, break_pattern='\n'):
    sentence_list = list(nlp(text).sents)
    sentence_list = sentence_list[: min(len(sentence_list), n_sentences)]

    # Add an additional pattern to split up the textract_n_sentences
    sent_list = []
    for sentence in sentence_list:
        sent_list += [sentence for sentence in sentence.text.split(break_pattern) if not sentence.isspace() and not len(sentence) == 0]

    return sent_list[: min(len(sent_list), n_sentences)]


In [169]:
c = extract_n_sentences(train_df.content.loc[111111114], en_nlp, 5)

In [170]:
c

['A U.S. judge on Wednesday ordered the release of an illegal immigrant who is among 47 Indonesians in New Hampshire challenging the Trump administration’s order to deport them.',
 'SPONSORED',
 'Rombot, part of a wave of Indonesian Christians who fled their country following deadly riots in 1998, learned of the policy change when he appeared for an Aug. 1 check-in with ICE and was arrested.',
 '“He walks out of the courthouse right now,” Chief U.S. District Judge Patti Saris said after concluding that Rombot’s detention violated his rights.',
 'He walked out of the U.S. District Court in Boston in his blue jail scrubs, without a chance to change into street clothes.']

In [157]:
c[0].text.split('\n')

['A U.S. judge on Wednesday ordered the release of an illegal immigrant who is among 47 Indonesians in New Hampshire challenging the Trump administration’s order to deport them.']

In [158]:
b.text.split('\n')

['A U.S. judge on Wednesday ordered the release of an illegal immigrant who is among 47 Indonesians in New Hampshire challenging the Trump administration’s order to deport them.']

In [ ]:
train_df.map(content.str.split(r'\n|.', 1, expand=True)

In [121]:
train_df.title + '\n' + train_df.content.str.split(r'\n|.', 1, expand=True)a

id
833042063    Far-left comedienne Chelsea Handler has admitt...
832959523    Those were the times…\nThe Times page 1 is of ...
833039623    Carload of crazies headed to the White House w...
833032367    But of course, this makes no difference to the...
814777937    This weekend in Manchester, England, Yellow Ve...
                                   ...                        
832908978    On Wednesday, Donald Trump was flying to a mil...
832910505    The Mueller circus cost the taxpayers millions...
832917532    Special counsel Robert Mueller will not recomm...
832913653    The Democratic presidential contenders are uni...
832917778    No new indictments will come from Special Coun...
Name: content, Length: 433, dtype: object

In [98]:
pipeline = ["lemmatizer"]
en_nlp = spacy.load("en_core_web_sm", disable=["tok2vec", "tagger", "parser", "ner", "attribute_ruler"])
en_nlp = spacy.load("en_core_web_sm")

lemmatize_rm_punct_and_stop = lambda str: [token.lemma_ for token in en_nlp(str) if not token.is_punct and not token.is_stop]

In [102]:
lemmatize_rm_punct_and_stop(train_df.title.iloc[10])

['Asteroid', 'Skim', 'Earth', 'day', 'Christmas', 'potentially', 'hazardous']

In [101]:
train_df.title.iloc[10]

'Asteroid Will Skim By Earth Days Before Christmas: It Is ‘Potentially Hazardous’'

## Generate TF-IDF features

In [113]:
tf_idf_vectorizer = TfidfVectorizer(tokenizer=lemmatize_rm_punct_and_stop, analyzer='word', max_df=0.95, min_df=5, max_features=None, use_idf=True, smooth_idf=True, sublinear_tf=False)

In [115]:
%timeit tf_idf_vectorizer = TfidfVectorizer(tokenizer=lemmatize_rm_punct_and_stop, analyzer='word', max_df=1, min_df=1, max_features=None, use_idf=True, smooth_idf=True, sublinear_tf=False)

1.9 µs ± 41.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [116]:
tf_idf_vectorizer.fit(train_df.raw_text.to_list())

/home/juanbermeo/anaconda3/envs/Framing/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


TfidfVectorizer(max_df=0.95, min_df=5,
                tokenizer=<function <lambda> at 0x7f85bcddac10>)

In [117]:
len(tf_idf_vectorizer.get_feature_names())

/home/juanbermeo/anaconda3/envs/Framing/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


4361

In [120]:
tf_idf_vectorizer.get_feature_names()

/home/juanbermeo/anaconda3/envs/Framing/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


['\n\n\n',
 '\n\n\n\n',
 '\n\n\n\n\n',
 ' ',
 '$',
 '+',
 '1',
 '1,000',
 '1.2',
 '10',
 '10,000',
 '100',
 '100,000',
 '11',
 '11th',
 '12',
 '120',
 '13',
 '14',
 '15',
 '150',
 '16',
 '17',
 '18',
 '19',
 '1960',
 '1961',
 '1965',
 '1976',
 '1979',
 '1980',
 '1982',
 '1983',
 '1990',
 '1990s',
 '1993',
 '1994',
 '1995',
 '1997',
 '1998',
 '19th',
 '1st',
 '2',
 '2,000',
 '20',
 '200',
 '2000',
 '2001',
 '2002',
 '2003',
 '2004',
 '2005',
 '2006',
 '2007',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '21',
 '22',
 '23',
 '24',
 '24/7',
 '25',
 '26',
 '27',
 '28',
 '29',
 '3',
 '30',
 '300',
 '31',
 '32',
 '32nd',
 '33',
 '34',
 '35',
 '36',
 '38',
 '39',
 '3d',
 '4',
 '40',
 '45',
 '46',
 '47',
 '48',
 '49',
 '5',
 '5,000',
 '50',
 '50,000',
 '500',
 '51',
 '53',
 '54',
 '55',
 '58',
 '5th',
 '6',
 '60',
 '600',
 '66',
 '7',
 '70',
 '700',
 '73',
 '75',
 '8',
 '80',
 '800',
 '84',
 '9',
 '9/11',
 '90',
 '91',
 '

In [118]:
tf_idf_vectorizer.transform(train_df.raw_text.iloc[10:12])

<2x4361 sparse matrix of type '<class 'numpy.float64'>'
	with 286 stored elements in Compressed Sparse Row format>